<a href="https://colab.research.google.com/github/hiiiiiiiiiieve/Rl_emobot/blob/main/emobot_beforeRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 실행 후 런타임 다시 시작

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# KoGPT2
#모듈 설치

!pip install --upgrade transformers
!pip uninstall -y pytorch-lightning
!pip install pytorch-lightning

#KoBERT
#모듈 설치

!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf' #허깅페이스
!pip install git+https://github.com/SKTBrain/KoBERT.git
!pip3 install kobert-transformers
!pip install boto3

### KoGPT2 / 필수

In [ ]:
#모듈 실행

import torch
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
# from pytorch_lightning.core.lightning import LightningModule #모듈 에러 발생해서 우선 보류
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re
import math
import numpy as np
import pandas as pd
import random
import re
import torch
import urllib.request
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast

RuntimeError: ignored

In [ ]:
#토큰 정의

Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

#토크나이저/모델 설정

# tokenizer_GPT = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
# koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
#             bos_token=BOS, eos_token=EOS, unk_token='<unk>',
#             pad_token=PAD, mask_token=MASK) #이게 기존 것, 아래는 tokenizer_GPT내용을 쓴 것
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
model_GPT = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

데이터셋

In [ ]:
#single-turn

import pandas as pd

wellness = pd.read_csv("/content/drive/MyDrive/study/EmoBot/우울증/1.dataset/wellness.csv", encoding = "cp949")
songys = pd.read_csv("/content/drive/MyDrive/study/EmoBot/우울증/1.dataset/songys.csv", encoding = "cp949")

wellness = wellness[['utterance(2차) ', 'response(공감)']]
wellness.rename(columns = {'utterance(2차) ' : 'utterance'}, inplace = True)
wellness.rename(columns = {'response(공감)' : 'response'}, inplace = True)
wellness = wellness.dropna(axis=0)

songys = songys[['Q', 'A']]
songys.rename(columns = {'Q' : 'utterance'}, inplace = True)
songys.rename(columns = {'A' : 'response'}, inplace = True)
songys = songys.dropna(axis=0)

single_data = pd.concat([wellness, songys], axis = 0)
single_data.to_csv("/content/drive/MyDrive/study/EmoBot/우울증/1.dataset/single_data") #필요 시 저장
single_data

single_data = single_data[['utterance', 'response']]
single_data

#랜덤 추출 (11897개)
single_data = single_data.sample(n = 11000)
single_data

In [ ]:
data1 = pd.read_csv("/content/drive/MyDrive/study/EmoBot/우울증/1.dataset/감성대화말뭉치(최종데이터)_Training_1turn.csv")
#랜덤 추출 (11897개)
data1 = data1.sample(n = 40000)

data2 = pd.read_csv("/content/drive/MyDrive/study/EmoBot/우울증/1.dataset/감성대화말뭉치(최종데이터)_Training_2turn.csv")
#랜덤 추출 (11897개)
data2 = data2.sample(n = 50000)

data3 = pd.read_csv("/content/drive/MyDrive/study/EmoBot/우울증/1.dataset/감성대화말뭉치(최종데이터)_Training_3turn.csv")
#랜덤 추출 (94325개)
data3 = data3.sample(n = 42000)

multiturn_data = pd.concat([data1, data2, data3], axis = 0)

multiturn_data.to_csv("/content/drive/MyDrive/study/EmoBot/우울증/1.dataset/감성대화말뭉치(최종데이터)_Training_final.csv", index=False)
multiturn_data= multiturn_data.drop("Unnamed: 0", axis=1)
multiturn_data

In [ ]:
emoji_data = pd.concat([single_data, multiturn_data], axis = 0)
emoji_data

chatbot 클래스 정의, 파라미터 설정 등

In [ ]:
# chatbotdata 만드는 클래스 정의하기

class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=256):  # 데이터셋의 전처리를 해주는 부분/max_len 원래 64
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["utterance"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["response"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

#collate_batch 클래스 정의하기

def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [ ]:
#학습세트 크기 정의하기

train_set = ChatbotDataset(emoji_data, max_len=256) #max_len 원래 50
#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=64, num_workers=0, shuffle=True, collate_fn=collate_batch,)

#cpu인지 아닌지 확인하는 코드

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

#아래 코드 해석 못함
model_GPT.to(device)
model_GPT.train()

In [ ]:
#학습 파라미터 설정

learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model_GPT.parameters(), lr=learning_rate)
epoch = 10 #multi = 10, single = 55
Sneg = -1e18

### KoGPT2 / 학습

In [ ]:
# # 학습

# print("start")
# for epoch in range(epoch):  # 반복 횟수 변수인 epoch가 정의되어야 합니다.
#     for batch_idx, samples in enumerate(train_dataloader):
#         optimizer.zero_grad()
#         token_ids, mask, label = samples
#         token_ids = token_ids.to(device)  # 입력 데이터를 GPU로 보냅니다.
#         mask = mask.to(device)  # 입력 데이터를 GPU로 보냅니다.
#         label = label.to(device)  # 입력 데이터를 GPU로 보냅니다.
#         out = model_GPT(token_ids)
#         out = out.logits
#         mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
#         mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
#         loss = criterion(mask_out.transpose(2, 1), label)
#         avg_loss = loss.sum() / mask.sum()
#         avg_loss.backward()
#         optimizer.step()
# print("end")

In [ ]:
# # 모델 상태 저장
# torch.save(model_GPT.state_dict(), '/content/drive/MyDrive/study/EmoBot/우울증/2.code/kogpt2_model_final.pth')

# # Optimizer 상태도 저장하려면 아래와 같이 추가합니다.
# torch.save(optimizer.state_dict(), '/content/drive/MyDrive/study/EmoBot/우울증/2.code/optimizer_final.pth')

### KoGPT2 / 불러오기

In [ ]:
# 모델 상태 불러오기 (wellness, corpus)
model_GPT = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')  # 모델 클래스를 생성
model_GPT.load_state_dict(torch.load('/content/drive/MyDrive/study/EmoBot/우울증/2.code/1. 저장 완료/kogpt2_model_final.pth'))
model_GPT.eval()  # 추론 모드로 전환

# Optimizer 상태를 불러오려면 아래와 같이 추가합니다.
optimizer.load_state_dict(torch.load('/content/drive/MyDrive/study/EmoBot/우울증/2.code/1. 저장 완료/optimizer_final.pth'))

In [ ]:
# # 두 가지 모델, 옵티마이저 병합

# import torch

# # 첫 번째 모델 및 옵티마이저 상태 불러오기
# model1_state = torch.load('/content/drive/MyDrive/study/EmoBot/우울증/2.code/1. 저장 완료/kogpt2_model.pth')
# optimizer1_state = torch.load('/content/drive/MyDrive/study/EmoBot/우울증/2.code/1. 저장 완료/optimizer.pth')

# # 두 번째 모델 및 옵티마이저 상태 불러오기
# model2_state = torch.load('/content/drive/MyDrive/study/EmoBot/우울증/2.code/kogpt2_model_final.pth')
# optimizer2_state = torch.load('/content/drive/MyDrive/study/EmoBot/우울증/2.code/optimizer_final.pth')

# # 두 모델 상태를 병합
# combined_model_state = {
#     'model_state_dict': model1_state,
#     'model_state_dict_2': model2_state
# }

# # 모델 상태를 별도의 파일로 저장
# torch.save(model_GPT.state_dict(), '/content/drive/MyDrive/study/EmoBot/우울증/2.code/1. 저장 완료/combined_model_single_multi.pth')

# # 두 옵티마이저 상태를 병합
# combined_optimizer_state = {
#     'optimizer_state_dict': optimizer1_state,
#     'optimizer_state_dict_2': optimizer2_state
# }

# # 옵티마이저 상태를 별도의 파일로 저장
# torch.save(optimizer.state_dict(), '/content/drive/MyDrive/study/EmoBot/우울증/2.code/1. 저장 완료/combined_optimizer_single_multi.pth')

### KoGPT2 / 대화 생성

In [ ]:
# # 모델을 GPU로 이동
# model_GPT.to('cuda:0')

# import csv

# # CSV 파일 경로
# csv_file = '/content/drive/MyDrive/study/EmoBot/우울증/3.result/kogpt2_result(wellness, songys, corpus).csv'

# # CSV 파일에 대화 데이터 저장을 위한 함수
# def save_conversation_to_csv(conversation_data):
#     with open(csv_file, 'a', newline='') as file:
#         fieldnames = ['kogpt2_utterance', 'kogpt2_response']
#         writer = csv.DictWriter(file, fieldnames=fieldnames)
#         if file.tell() == 0:  # 파일이 비어 있는 경우에만 헤더를 작성
#             writer.writeheader()
#         writer.writerow(conversation_data)

# # 대화 데이터 초기화
# conversation_data = {
#     'kogpt2_utterance': '',
#     'kogpt2_response': '',
# }

# # 대화 예시 추출
# print("안녕하세요!\n")
# name = input("사용자님의 성함을 말씀해주세요:")
# print(f'\n{name}님 안녕하세요.\nChatbot과 함께 대화를 나눠보아요.\n대화 종료를 원할 경우, quit를 입력해주세요.\n')

# while True:
#     kogpt2_utterance = input("user > ").strip()
#     if kogpt2_utterance == "quit":
#         print(f'{name}님 대화를 종료하겠습니다.')
#         break

#     conversation_data['kogpt2_utterance'] = kogpt2_utterance

#     kogpt2_response = ""
#     while True:
#         input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode("<usr>"+ kogpt2_utterance + "<unused1>" + "<sys>" + kogpt2_response)).unsqueeze(dim=0).to(device)
#         pred = model_GPT(input_ids)
#         pred = pred.logits
#         gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]
#         if gen == "</s>":
#             break
#         kogpt2_response += gen.replace("▁", " ")
#         if "00" in kogpt2_response:
#             kogpt2_response = kogpt2_response.replace("00", "사용자")

#     conversation_data['kogpt2_response'] = kogpt2_response

#     print(f"Chatbot > {kogpt2_response.strip()}\n")
#     print("\n")
#     # 대화 데이터를 CSV 파일에 추가
#     save_conversation_to_csv(conversation_data)

# # CSV 파일 닫기

In [ ]:
# #csv 내 데이터 있는지 확인 코드

# import pandas as pd


# if emoji_data['utterance'].str.contains('더 후회하기 전에 사과하세요').any():
#     print("데이터가 존재합니다.")
# else:
#     print("데이터가 존재하지 않습니다.")

In [ ]:
import torch
import csv
import random  # 랜덤 샘플링을 위한 라이브러리 추가

# 모델을 GPU로 이동
model_GPT.to('cuda:0')

# CSV 파일 경로
csv_file = '/content/drive/MyDrive/study/EmoBot/우울증/3.result/kogpt2_result_crawling.csv'

# CSV 파일에 대화 데이터 저장을 위한 함수
def save_conversation_to_csv(conversation_data):
    with open(csv_file, 'a', newline='') as file:
        fieldnames = ['kogpt2_utterance', 'kogpt2_response']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        if file.tell() == 0:  # 파일이 비어 있는 경우에만 헤더를 작성
            writer.writeheader()
        writer.writerow(conversation_data)

# emoji_data에서 랜덤으로 1000개 추출
random_samples = emoji_data['utterance'].dropna().sample(1000)

# 대화 예시 추출
print("안녕하세요!\n")
name = input("사용자님의 성함을 말씀해주세요:")
print(f'\n{name}님 안녕하세요.\nChatbot과 함께 대화를 나눠보아요.\n대화 종료를 원할 경우, quit를 입력해주세요.\n')

for utterance_value in random_samples:  # 랜덤으로 추출한 1000개에 대해 반복
    conversation_data = {
        'kogpt2_utterance': utterance_value,
        'kogpt2_response': '',
    }

    kogpt2_response = ""
    while True:
        input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode("<usr>"+ utterance_value + "<unused1>" + "<sys>" + kogpt2_response)).unsqueeze(dim=0).to(device)
        pred = model_GPT(input_ids)
        pred = pred.logits
        gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]
        if gen == "</s>":
            break
        kogpt2_response += gen.replace("▁", " ")
        if "00" in kogpt2_response:
            kogpt2_response = kogpt2_response.replace("00", "사용자")

    conversation_data['kogpt2_response'] = kogpt2_response

    # Chatbot의 답변과 사용자의 입력을 출력
    print(f"user> {utterance_value}")
    print(f"Chatbot > {kogpt2_response.strip()}\n")
    print("\n")

    # 대화 데이터를 CSV 파일에 추가
    save_conversation_to_csv(conversation_data)

print(f'{name}님 대화를 종료하겠습니다.')

# **KoBERT**


*   학습은 corpus로 진행하기
*   입력은 KoGPT2의 답변으로 하기



In [ ]:
#모듈 실행

import gluonnlp as nlp
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
#cpu 여부 확인

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

#토크나이저/모델/보캡 설정

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
import pandas as pd

corpus_emotion = pd.read_excel("/content/drive/MyDrive/study/EmoBot/우울증/1.dataset/한국어단발성대화데이터셋.xlsx")
corpus_emotion

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
...,...,...,...,...,...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오,NaN,NaN,NaN,NaN,NaN
38590,재미가 없으니 망하지,혐오,NaN,NaN,NaN,NaN,NaN
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,NaN,NaN,NaN,NaN,NaN
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오,NaN,NaN,NaN,NaN,NaN


In [ ]:
corpus_emotion['Emotion'].value_counts()

행복    6037
놀람    5898
분노    5665
공포    5468
혐오    5429
슬픔    5267
중립    4830
Name: Emotion, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

#테스트 정확도가 너무 낮아서 7:3 비율로 진행해보기
train_ratio = 0.7
test_ratio = 0.3

# 각각의 감정에 대한 인덱스를 추출합니다.
emotion_행복 = corpus_emotion[corpus_emotion['Emotion'] == '행복']
emotion_놀람 = corpus_emotion[corpus_emotion['Emotion'] == '놀람']
emotion_분노 = corpus_emotion[corpus_emotion['Emotion'] == '분노']
emotion_공포 = corpus_emotion[corpus_emotion['Emotion'] == '공포']
emotion_혐오 = corpus_emotion[corpus_emotion['Emotion'] == '혐오']
emotion_슬픔 = corpus_emotion[corpus_emotion['Emotion'] == '슬픔']
emotion_중립 = corpus_emotion[corpus_emotion['Emotion'] == '중립']

train_emotion_행복, test_emotion_행복 = train_test_split(emotion_행복, test_size=test_ratio, random_state=42)
train_emotion_놀람, test_emotion_놀람 = train_test_split(emotion_놀람, test_size=test_ratio, random_state=42)
train_emotion_분노, test_emotion_분노 = train_test_split(emotion_분노, test_size=test_ratio, random_state=42)
train_emotion_공포, test_emotion_공포 = train_test_split(emotion_공포, test_size=test_ratio, random_state=42)
train_emotion_혐오, test_emotion_혐오 = train_test_split(emotion_혐오, test_size=test_ratio, random_state=42)
train_emotion_슬픔, test_emotion_슬픔 = train_test_split(emotion_슬픔, test_size=test_ratio, random_state=42)
train_emotion_중립, test_emotion_중립 = train_test_split(emotion_중립, test_size=test_ratio, random_state=42)

dataset_train = pd.concat([train_emotion_행복, train_emotion_놀람, train_emotion_분노, train_emotion_공포, train_emotion_혐오, train_emotion_슬픔, train_emotion_중립], axis = 0)
dataset_test = pd.concat([test_emotion_행복, test_emotion_놀람, test_emotion_분노, test_emotion_공포, test_emotion_혐오, test_emotion_슬픔, test_emotion_중립], axis = 0)


# 훈련 데이터와 테스트 데이터의 감정 비율을 확인합니다.
print("훈련 데이터의 감정 비율:")
print(dataset_train['Emotion'].value_counts())
print("\n테스트 데이터의 감정 비율:")
print(dataset_test['Emotion'].value_counts())

훈련 데이터의 감정 비율:
행복    4225
놀람    4128
분노    3965
공포    3827
혐오    3800
슬픔    3686
중립    3381
Name: Emotion, dtype: int64

테스트 데이터의 감정 비율:
행복    1812
놀람    1770
분노    1700
공포    1641
혐오    1629
슬픔    1581
중립    1449
Name: Emotion, dtype: int64


In [ ]:
# depressed_data.loc[(depressed_data['감정'] == "지속되는우울한기분(우울감)"), '감정'] = 0 와 같은 형태로 라벨링
#'분노', '기쁨', '불안', '당황', '슬픔', '상처' (감성 말뭉치 기준)

dataset_train.loc[(dataset_train['Emotion'] == '행복'), 'Emotion'] = 0
dataset_train.loc[(dataset_train['Emotion'] == '놀람'), 'Emotion'] = 1
dataset_train.loc[(dataset_train['Emotion'] == '분노'), 'Emotion'] = 2
dataset_train.loc[(dataset_train['Emotion'] == '공포'), 'Emotion'] = 3
dataset_train.loc[(dataset_train['Emotion'] == '혐오'), 'Emotion'] = 4
dataset_train.loc[(dataset_train['Emotion'] == '슬픔'), 'Emotion'] = 5
dataset_train.loc[(dataset_train['Emotion'] == '중립'), 'Emotion'] = 6

dataset_test.loc[(dataset_test['Emotion'] == '행복'), 'Emotion'] = 0
dataset_test.loc[(dataset_test['Emotion'] == '놀람'), 'Emotion'] = 1
dataset_test.loc[(dataset_test['Emotion'] == '분노'), 'Emotion'] = 2
dataset_test.loc[(dataset_test['Emotion'] == '공포'), 'Emotion'] = 3
dataset_test.loc[(dataset_test['Emotion'] == '혐오'), 'Emotion'] = 4
dataset_test.loc[(dataset_test['Emotion'] == '슬픔'), 'Emotion'] = 5
dataset_test.loc[(dataset_test['Emotion'] == '중립'), 'Emotion'] = 6

#리스트 생성

dataset_train_list = []
for ques1, label1 in zip(dataset_train['Sentence'], dataset_train['Emotion']):
    data_item1 = []
    data_item1.append(ques1)
    data_item1.append(str(label1))
    dataset_train_list.append(data_item1)

dataset_test_list = []
for ques2, label2 in zip(dataset_test['Sentence'], dataset_test['Emotion']):
    data_item2 = []
    data_item2.append(ques2)
    data_item2.append(str(label2))
    dataset_test_list.append(data_item2)

In [ ]:
print(len(dataset_train_list))
print(len(dataset_test_list))

27012
11582


In [ ]:
#데이터 분할

# from sklearn.model_selection import train_test_split
# dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)
dataset_train = dataset_train_list
dataset_test = dataset_test_list

#파라미터 설정

max_len = 256 #원래 64
batch_size = 128 #loss 줄이기 위해 높여보기
warmup_ratio = 0.1
num_epochs = 55 #loss 줄이기 위해 높여보기
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5 #loss 줄이기 위해 낮춰보기

In [ ]:
#bert 데이터셋 만드는 클래스 정의하기

class BERTDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_len, pad=True, pair=False):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair
        self.dataset = dataset

    def __getitem__(self, idx):
        if self.pair:
            text = self.dataset[idx][0][0]
            text_pair = self.dataset[idx][0][1]
        else:
            text = self.dataset[idx][0]

        inputs = self.tokenizer.encode_plus(
            text,
            text_pair if self.pair else None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=self.pad,
            truncation=False
        )

        item = {
            'input_ids': torch.tensor(inputs['input_ids']),
            'attention_mask': torch.tensor(inputs['attention_mask']),
            'token_type_ids': torch.tensor(inputs['token_type_ids'])
        }

        item['labels'] = int(self.dataset[idx][1])

        # 패딩된 시퀀스/길이와 타입에 대한 내용/어텐션 마스크 시퀀스 세 가지 배열을 얻을 수 있도록 함
        padded_seq = torch.tensor(inputs['input_ids']).numpy()
        len_and_type = list(torch.tensor(inputs['token_type_ids']).shape) #dtype은 출력 안됨
        attention_mask_seq = torch.tensor(inputs['attention_mask']).numpy()

        return np.array(padded_seq), np.array(len_and_type), np.array(attention_mask_seq), item['labels']

    def __len__(self):
        return len(self.dataset)

# 데이터셋 생성
train_dataset = BERTDataset(dataset_train, tokenizer, max_len, True, True)
test_dataset = BERTDataset(dataset_test, tokenizer, max_len, True, True)

# 데이터로더 생성
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

#학습 데이터셋 예시 추출해보기

print(train_dataset[0])

#bert 분류하는 클래스 정의하기

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

#모델 정의

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# 아래 코드 해석 못함

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

(array([  2, 529,   3, 589,   3,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,  

RuntimeError: ignored

In [ ]:
# train_history = []
# test_history = []
# loss_history = []

# for e in range(num_epochs):
#     train_acc = 0.0
#     test_acc = 0.0
#     model.train()
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_loader)):
#         optimizer.zero_grad()
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)

#         # print(label.shape, out.shape)
#         loss = loss_fn(out, label)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
#         optimizer.step()
#         scheduler.step()  # Update learning rate schedule
#         train_acc += calc_accuracy(out, label)
#         if batch_id % log_interval == 0:
#             print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
#             train_history.append(train_acc / (batch_id+1))
#             loss_history.append(loss.data.cpu().numpy())
#     print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
#     # train_history.append(train_acc / (batch_id+1))

#     # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
#     # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
#     model.eval()
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_loader)):
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length = valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         test_acc += calc_accuracy(out, label)
#     print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
#     test_history.append(test_acc / (batch_id+1))

In [ ]:
# # 모델 상태 저장
# torch.save(model.state_dict(), '/content/drive/MyDrive/study/EmoBot/우울증/2.code/kobert_classifier_model.pth')

# # Optimizer 상태도 저장하려면 아래와 같이 추가합니다.
# torch.save(optimizer.state_dict(), '/content/drive/MyDrive/study/EmoBot/우울증/2.code/optimizer_KoBERT.pth')

# # train_history, test_history, loss_history 저장
# import pickle

# with open('/content/drive/MyDrive/study/EmoBot/우울증/2.code/train_history.pkl', 'wb') as f:
#     pickle.dump(train_history, f)

# with open('/content/drive/MyDrive/study/EmoBot/우울증/2.code/test_history.pkl', 'wb') as f:
#     pickle.dump(test_history, f)

# with open('/content/drive/MyDrive/study/EmoBot/우울증/2.code/loss_history.pkl', 'wb') as f:
#     pickle.dump(loss_history, f)

In [ ]:
# 모델 상태 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/study/EmoBot/우울증/2.code/1. 저장 완료/kobert_classifier_model.pth'))
model.eval()  # 추론 모드로 전환

# Optimizer 상태를 불러오려면 아래와 같이 추가합니다.
optimizer.load_state_dict(torch.load('/content/drive/MyDrive/study/EmoBot/우울증/2.code/1. 저장 완료/optimizer_KoBERT.pth'))

# train_history, test_history, loss_history 불러오기
import pickle

with open('/content/drive/MyDrive/study/EmoBot/우울증/2.code/1. 저장 완료/train_history.pkl', 'rb') as f:
    train_history = pickle.load(f)

with open('/content/drive/MyDrive/study/EmoBot/우울증/2.code/1. 저장 완료/test_history.pkl', 'rb') as f:
    test_history = pickle.load(f)

with open('/content/drive/MyDrive/study/EmoBot/우울증/2.code/1. 저장 완료/loss_history.pkl', 'rb') as f:
    loss_history = pickle.load(f)

In [ ]:
import random

# 감정 이모티콘 데이터
emoticon_label = {
    'happy': ['😄', '😃', '😁', '😆', '😉', '😊', '😋'],
    'surprised': ['😲', '😮', '😯'],
    'angry': ['😡', '😠', '🤬', '😤'],
    'fear': ['😦', '😧', '😨'],
    'disgust': ['😫', '😩', '😣'],
    'sad': ['😢', '😞', '😔', '😢', '☹️', '🙁', '🥺', '😭'],
    'neutral': ['😗', '🙂', '😙', '😚']
}

#감정 예측하는 함수 정의

def emoticon_predict(predict_sentence):

    #함수 밖에서 정의
    # emoticon_label = {
    #     'happy': ['😄', '😃', '😁', '😆', '😉', '😊', '😋'],
    #     'surprised': ['😲', '😮', '😯'],
    #     'angry': ['😡', '😠', '🤬', '😤'],
    #     'fear': ['😦', '😧', '😨'],
    #     'disgust': ['😫', '😩', '😣'],
    #     'sad': ['😢', '😞', '😔', '😢', '☹️', '🙁', '🥺', '😭'],
    #     'neutral': ['😗', '🙂', '😙', '😚']
    # }

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, tokenizer, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        emoticon_add = []
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()


            #분노 기쁨 슬픔만 진행하는 것도 고려해보기
            #에크만 참고해서 7가지로 분류
            if np.argmax(logits) == 0:
                # emotion = "행복이"
                emoticon_select = random.choice(emoticon_label['happy'])
                emoticon_add.append(emoticon_select)
            elif np.argmax(logits) == 1:
                # emotion = "놀람이"
                emoticon_select = random.choice(emoticon_label['surprised'])
                emoticon_add.append(emoticon_select)
            elif np.argmax(logits) == 2:
                # emotion = "분노가"
                emoticon_select = random.choice(emoticon_label['angry'])
                emoticon_add.append(emoticon_select)
            elif np.argmax(logits) == 3:
                # emotion = "공포가"
                emoticon_select = random.choice(emoticon_label['fear'])
                emoticon_add.append(emoticon_select)
            elif np.argmax(logits) == 4:
                # emotion = "혐오가"
                emoticon_select = random.choice(emoticon_label['disgust'])
                emoticon_add.append(emoticon_select)
            elif np.argmax(logits) == 5:
                # emotion = "슬픔이"
                emoticon_select = random.choice(emoticon_label['sad'])
                emoticon_add.append(emoticon_select)
            elif np.argmax(logits) == 6:
                # emotion = "중립이"
                emoticon_select = random.choice(emoticon_label['neutral'])
                emoticon_add.append(emoticon_select)

            return emoticon_add

In [ ]:
import torch
import csv
import random

# 모델을 GPU로 이동
model_GPT.to('cuda:0')

# CSV 파일 경로
csv_file = '/content/drive/MyDrive/study/EmoBot/우울증/3.result/emobot_result_crawling.csv'

# CSV 파일에 대화 데이터 저장을 위한 함수
def save_conversation_to_csv(conversation_data_emobot):
    with open(csv_file, 'a', newline='') as file:
        fieldnames = ['emobot_utterance', 'emobot_response']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        if file.tell() == 0:  # 파일이 비어 있는 경우에만 헤더를 작성
            writer.writeheader()
        writer.writerow(conversation_data_emobot)

# 대화 데이터 초기화
conversation_data_emobot = {
    'emobot_utterance': '',
    'emobot_response': '',
}

# 경고 무시 설정
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# 대화 예시 추출
print("안녕하세요!\n")
name = str(input("사용자님의 성함을 말씀해주세요:"))
print("\n")
print(f'{name}님 안녕하세요.😊\nEmoBot🤖과 함께 대화를 나눠보아요.\n대화 종료를 원할 경우, quit를 입력해주세요.')
print("\n")

# emoji_data에서 랜덤으로 1000개 추출
random_samples = emoji_data['utterance'].dropna().sample(1000)

with torch.no_grad():
    for emobot_utterance in random_samples:  # 랜덤으로 추출한 1000개에 대해 반복
        all_emobot_conversation.append(emobot_utterance)
        if emobot_utterance == "quit":
            print(f'{name}님 대화를 종료하겠습니다.🙋')
            break
        conversation_data_emobot['emobot_utterance'] = emobot_utterance

        emobot_response = ""
        while True:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode("<usr>"+ emobot_utterance + "<unused1>" + "<sys>" + emobot_response)).unsqueeze(dim=0).to(device)
            pred = model_GPT(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]
            if gen == "</s>":
                break
            emobot_response += gen.replace("▁", " ")
            if "00" in emobot_response:
                emobot_response = emobot_response.replace("00", "사용자")

        emoticons = emoticon_predict(emobot_response)
        if "감사" in emobot_response:
            emoticons = [random.choice(emoticon_label['happy'])]
        emoticons_str = ' '.join(emoticons)
        response_with_emoticons = emobot_response + emoticons_str
        all_emobot_conversation.append(response_with_emoticons)
        conversation_data_emobot['emobot_response'] = response_with_emoticons

        print("EmoBot > {}".format(response_with_emoticons.strip()))
        print("\n")
        # 대화 데이터를 CSV 파일에 추가
        save_conversation_to_csv(conversation_data_emobot)

# CSV 파일 닫기

In [ ]:
# 아래 코드는 어떤 문장이든지 이모티콘 붙임
# 정확도가 낮은 경우가 있음

# responses with emojis

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# 모델을 GPU로 이동
model_GPT.to('cuda:0')

import csv

# CSV 파일 경로
csv_file = '/content/drive/MyDrive/study/EmoBot/우울증/3.result/emobot_result(wellness, songys, corpus).csv'

# CSV 파일에 대화 데이터 저장을 위한 함수
def save_conversation_to_csv(conversation_data_emobot):
    with open(csv_file, 'a', newline='') as file:
        fieldnames = ['emobot_utterance', 'emobot_response']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        if file.tell() == 0:  # 파일이 비어 있는 경우에만 헤더를 작성
            writer.writeheader()
        writer.writerow(conversation_data_emobot)

# 대화 데이터 초기화
conversation_data_emobot = {
    'emobot_utterance': '',
    'emobot_response': '',
}

# 경고 무시 설정
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

#대화 예시 추출하기
#KoGPT2 + Emoji
print('안녕하세요!\n')
name = str(input("사용자님의 성함을 말씀해주세요:"))
print("\n")
print(f'{name}님 안녕하세요.😊\nEmoBot🤖과 함께 대화를 나눠보아요.\n대화 종료를 원할 경우, quit를 입력해주세요.')
print("\n")
all_emobot_conversation = []

with torch.no_grad():
    while True:
        emobot_utterance = input("user > ").strip()
        all_emobot_conversation.append(emobot_utterance)
        if emobot_utterance == "quit":
          print(f'{name}님 대화를 종료하겠습니다.🙋')
          break
        conversation_data_emobot['emobot_utterance'] = emobot_utterance

        emobot_response = ""
        while True:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode("<usr>"+ emobot_utterance + "<unused1>" + "<sys>" + emobot_response)).unsqueeze(dim=0).to(device)
            pred = model_GPT(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]
            if gen == "</s>":
                break
            emobot_response += gen.replace("▁", " ")
            if "00" in emobot_response:
              emobot_response = emobot_response.replace("00", "사용자")
            # if "감사" in emobot_response:
            #   emobot_response = "감사합니다 😊"
        emoticons = emoticon_predict(emobot_response)  # 감정 이모티콘을 얻어옴
        if "감사" in emobot_response:
            # "감사"가 포함된 경우, "happy" 감정에 해당하는 이모티콘 중 하나를 랜덤으로 선택
            emoticons = [random.choice(emoticon_label['happy'])]
        else:
            # "감사"가 포함되지 않은 경우, 앞서 얻어온 감정 이모티콘 사용
            emoticons_str = ' '.join(emoticons)
        emoticons_str = ' '.join(emoticons)  # 이모티콘을 문자열로 변환
        response_with_emoticons = emobot_response + emoticons_str  # 이모티콘을 응답에 추가
        all_emobot_conversation.append(response_with_emoticons)
        conversation_data_emobot['emobot_response'] = response_with_emoticons

        print("EmoBot > {}".format(response_with_emoticons.strip()))
        print("\n")
        # 대화 데이터를 CSV 파일에 추가
        save_conversation_to_csv(conversation_data_emobot)

# CSV 파일 닫기

In [ ]:
import torch
import pandas as pd
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
import torch.nn as nn
import os
import numpy as np

# KoBERT 토크나이저와 모델 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)

class RegressionNet(nn.Module):
    def __init__(self, input_dim):
        super(RegressionNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, input_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

def predict_vad(dlg_emotion, label_emotion, dlg_vad):
    # model_name = f"{dlg_emotion}_to_{label_emotion}_regression.pt"
    model_path = os.path.join('/content/drive/MyDrive/study/EmoBot/우울증/2.code/kobert_classifier_model.pth')

    model = RegressionNet(input_dim=3)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    with torch.no_grad():
        input_tensor = torch.tensor(np.array([dlg_vad]), dtype=torch.float32)
        predicted_vad = model(input_tensor).numpy()[0]

    return predicted_vad

def get_vad(dlg_emotion, label_emotion, dlg_vad):
    predicted_vad = predict_vad(dlg_emotion, label_emotion, dlg_vad)
    return predicted_vad

def vad_regression(emo):
    # 이 함수는 감정 벡터(emo)를 입력으로 받아 VAD 값을 예측하는 함수입니다.
    # 실제로는 VAD 값을 예측하기 위한 회귀 모델을 사용해야 합니다.
    # 아래는 더미 값으로 VAD를 생성하는 예제입니다.

    # 각 VAD 차원을 더미 값으로 설정
    valence = 0.7
    arousal = 0.6
    dominance = 0.8

    return [valence, arousal, dominance]

# SimpleReg 모델 생성
model = RegressionNet()

# 학습된 가중치 불러오기
model.load_state_dict(torch.load('/content/drive/MyDrive/study/EmoBot/우울증/2.code/kobert_classifier_model.pth'))

# 나머지 코드 실행
csv_file_path = '/content/drive/MyDrive/study/EmoBot/우울증/3.result/kogpt2_result_crawling.csv'

# CSV 파일 불러오기
df2 = pd.read_csv(csv_file_path)

# 리워드 함수 정의 (with emoji)
def get_reward_for_response(response, prev_response, correct_emotion, dlg_emotion, correct_emotion_accuracy=0.8):
    # 1. 답변 길이에 대한 리워드 (0~10점)
    length_reward = max(0, min(10, 10 - abs(len(response) - 20) / 2))

    # 2. 이전과 동일한 답변에 대한 패널티 (0~10점)
    repetition_penalty = max(0, min(10, 10 - int(response == prev_response)))

    # 3. 감정 분류 이모티콘에 대한 리워드 (0~10점)
    correct_emotion_reward = max(0, min(10, 10 * correct_emotion_accuracy * correct_emotion))

    # 4. VAD 예측에 대한 리워드 (0~10점)
    # 아래 코드는 더미 값으로 VAD를 가져오는 부분입니다.
    dlg_vad = vad_regression(dlg_emotion)
    pre_vad = get_vad(dlg_emotion, correct_emotion, dlg_vad)
    vad_reward = max(0, min(10, 10 * np.dot(pre_vad, dlg_vad)))

    # 전체 리워드 계산 (총 40점 만점)
    total_reward = length_reward + repetition_penalty + correct_emotion_reward + vad_reward

    return total_reward

# 초기값 설정
prev_response = None  # 이전 답변이 없을 경우 None으로 설정

# 리워드 계산 및 출력
for index, row in df2.iterrows():
    response = row['kogpt2_response']
    dlg_emotion = row['dlg_emotion']

    # 리워드 계산
    reward = get_reward_for_response(response, prev_response, correct_emotion, dlg_emotion)

    # 결과 출력 (혹은 저장)
    print(f"Response: {response}, Total Reward: {reward}")

    # 다음 반복을 위해 이전 답변 업데이트
    prev_response = response


In [ ]:
import torch
import pandas as pd
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
import torch.nn as nn
import os
import numpy as np

# KoBERT 토크나이저와 모델 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)

class RegressionNet(nn.Module):
    def __init__(self, input_dim):
        super(RegressionNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, input_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

def predict_vad(dlg_emotion, label_emotion, dlg_vad):
    # model_name = f"{dlg_emotion}_to_{label_emotion}_regression.pt"
    model_path = os.path.join('/content/drive/MyDrive/study/EmoBot/우울증/2.code/kobert_classifier_model.pth')

    model = RegressionNet(input_dim=3)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    with torch.no_grad():
        input_tensor = torch.tensor(np.array([dlg_vad]), dtype=torch.float32)
        predicted_vad = model(input_tensor).numpy()[0]

    return predicted_vad

def get_vad(dlg_emotion, label_emotion, dlg_vad):
    predicted_vad = predict_vad(dlg_emotion, label_emotion, dlg_vad)
    return predicted_vad

def vad_regression(emo):
    # 이 함수는 감정 벡터(emo)를 입력으로 받아 VAD 값을 예측하는 함수입니다.
    # 실제로는 VAD 값을 예측하기 위한 회귀 모델을 사용해야 합니다.
    # 아래는 더미 값으로 VAD를 생성하는 예제입니다.

    # 각 VAD 차원을 더미 값으로 설정
    valence = 0.7
    arousal = 0.6
    dominance = 0.8

    return [valence, arousal, dominance]

# SimpleReg 모델 생성
model = RegressionNet()

# 학습된 가중치 불러오기
model.load_state_dict(torch.load('/content/drive/MyDrive/study/EmoBot/우울증/2.code/kobert_classifier_model.pth'))

# 나머지 코드 실행
csv_file_path = '/content/drive/MyDrive/study/EmoBot/우울증/3.result/emoobot_result_crawling.csv'

# CSV 파일 불러오기
df2 = pd.read_csv(csv_file_path)

# 리워드 함수 정의 (with emoji)
def get_reward_for_response(response, prev_response, correct_emotion, dlg_emotion, correct_emotion_accuracy=0.8):
    # 1. 답변 길이에 대한 리워드 (0~10점)
    length_reward = max(0, min(10, 10 - abs(len(response) - 20) / 2))

    # 2. 이전과 동일한 답변에 대한 패널티 (0~10점)
    repetition_penalty = max(0, min(10, 10 - int(response == prev_response)))

    # 3. 감정 분류 이모티콘에 대한 리워드 (0~10점)
    correct_emotion_reward = max(0, min(10, 10 * correct_emotion_accuracy * correct_emotion))

    # 4. VAD 예측에 대한 리워드 (0~10점)
    # 아래 코드는 더미 값으로 VAD를 가져오는 부분입니다.
    dlg_vad = vad_regression(dlg_emotion)
    pre_vad = get_vad(dlg_emotion, correct_emotion, dlg_vad)
    vad_reward = max(0, min(10, 10 * np.dot(pre_vad, dlg_vad)))

    # 전체 리워드 계산 (총 40점 만점)
    total_reward = length_reward + repetition_penalty + correct_emotion_reward + vad_reward

    return total_reward

# 초기값 설정
prev_response = None  # 이전 답변이 없을 경우 None으로 설정

# 리워드 계산 및 출력
for index, row in df2.iterrows():
    response = row['emobot_response']
    dlg_emotion = row['dlg_emotion']

    # 리워드 계산
    reward = get_reward_for_response(response, prev_response, correct_emotion, dlg_emotion)

    # 결과 출력 (혹은 저장)
    print(f"Response: {response}, Total Reward: {reward}")

    # 다음 반복을 위해 이전 답변 업데이트
    prev_response = response


In [ ]:
# #우선 보류

# import pandas as pd

# # CSV 파일 경로
# csv_file_path = '/content/drive/MyDrive/study/EmoBot/우울증/3.result/emoobot_result_crawling.csv'

# # CSV 파일 불러오기
# df2 = pd.read_csv(csv_file_path)

# # 첫 번째 행을 헤더로 설정
# df2

# # 리워드 함수 정의 (with emoji)
# def get_reward_for_response(response, prev_response, correct_emotion):
#     # 1. 답변 길이에 대한 리워드 (0~10점)
#     length_reward = max(0, min(10, 10 - abs(len(response) - 20) / 2))

#     # 2. 이전과 동일한 답변에 대한 패널티 (0~10점)
#     repetition_penalty = max(0, min(10, 10 - int(response == prev_response)))

#     # 3. 감정 분류 이모티콘에 대한 리워드 (0~10점)
#     correct_emotion_reward = max(0, min(10, 10 * correct_emotion))

#     # 전체 리워드 계산 (총 30점 만점)
#     total_reward = length_reward + repetition_penalty + correct_emotion_reward

#     return total_reward

# # 초기값 설정
# prev_response = None  # 이전 답변이 없을 경우 None으로 설정
# correct_emotion = False  # 실제 감정 분류 결과에 따라 수정

# # 리워드 계산 및 출력
# for index, row in df2.iterrows():
#     response = row['emobot_response']

#     # 리워드 계산
#     reward = get_reward_for_response(response, prev_response, correct_emotion)

#     # 결과 출력 (혹은 저장)
#     print(f"Response: {response}, Total Reward: {reward}")

#     # 다음 반복을 위해 이전 답변 업데이트
#     prev_response = response